In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [60]:
import cv2
import numpy as np
import random

# Set the scale 
scale = 1000 # Actual length of the scale
scale_px = 114 # Length of the scale in pixels

# Set the HoughCircles function parameters

dp = 1 # Inverse ratio of the resolution of the accumulator array to the resolution of the input image
minDist = 250 # Minimum distance between two circles
param1 = 44 # Higher threshold value for the edge detector from gradient between neighbouring pixels; the higher the value the fewer edges are detected
param2 = 12 # Accumulator threshold for the circle centers; controls the minimum number of edge pixels that must support a candidate circle
rmin = 15 # Minimum detected radius
rmax = 25 # Maximum detected radius

blur = 47 # Gausian blur level

# Load the image, cut out the bottom part, and convert to grayscale
img_name = "NIPS315.tif"
img_loc = "C:/Users/brtkm/Desktop/NiPS3 4A/"

incorrect_image_name = True

while incorrect_image_name:
  
  try:
    img = cv2.imread(img_loc + img_name)
    incorrect_image_name = False
  except:
    print("wrong image name")

img = img[0:715, 0:1024]
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Blur the image to reduce noise
blurred_img = cv2.GaussianBlur(gray_img, (blur, blur), 0)

# Define the alpha and beta for brightness and contrast corection
#alpha = 2.5 # Contrast control
#beta = -120 # Brightness control
#img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

# Detect circles using HoughCircles function
def detect_and_draw_circles(blurred_img, dp, minDist, param1, param2, rmin, img, print_accuracy):
  circles = cv2.HoughCircles(blurred_img, cv2.HOUGH_GRADIENT, dp, minDist, param1, param2, rmin, maxRadius=rmax)
  
  # If no circles are detected, print a message and exit the program
  if circles is None:
      print('No circles detected.')
      exit()
  else:
      # Convert the (x, y) coordinates and radius of the circles to integers
      circles = np.round(circles[0, :]).astype("int")
      
      # Loop through all detected circles and draw them on the original image
      for (x, y, r) in circles:
          cv2.circle(img, (x, y), r, (98, 33, 135), 1)
          
          # Put a text specifying the radius
          scaled = (r+2)*scale/scale_px
          size = 'r=' + str("%.1f" % scaled) + 'nm'
          cv2.putText(img,size,(x-45,y+r+30), cv2.FONT_HERSHEY_PLAIN, 1,(226,228,229),1,cv2.LINE_AA)
      
          # Draw a small circle at the center of the detected circle
          cv2.circle(img, (x, y), 3, (169, 169, 169), -1)
      
          # Print the radius of the detected circle
          # print(f"Circle with radius {r} detected at ({x}, {y})")
      
      error = 2*scale/scale_px
      if print_accuracy:
        print('Accuracy of the radii ' + u"\u00B1 " + str("%.1f" % error))
  
      # Show the original image with the detected circles
      cv2.imshow("Detected antennas", img)
      cv2.waitKey(0)
      cv2.destroyAllWindows()

detect_and_draw_circles(blurred_img, dp, minDist, param1, param2, rmin, img, True)

def save_image(img):

  try:
    numbers = open("SEM_img_analysis_config.txt", 'r')
    number = numbers.readline()
    numbers.close()
  except:
    print("no numbers file")
    numbers = open("SEM_img_analysis_config.txt", "w+")
    numbers.close()
    number = "1 "

  save = input("Save the image? (y/n): ")
  if save:
    cv2.imwrite("image" + number[0] + ".jpg", img)
    new_num = int(number[0]) + 1
    numbers = open("SEM_img_analysis_config.txt", 'w')
    numbers.write(str(new_num))
    numbers.close()

save_image(img)

running = input("Run again with different parameters? (y/n): ")

previous_dps = []
previous_dps.append(dp)

while running == 'y':
  #dp: 1-1.5
  #param1: 52-100 
  #param2: 25-50

  while dp in previous_dps:
    dp = random.uniform(0.9, 1.6)
    dp = round(dp, 1)
    
  previous_dps.append(dp)

  img = cv2.imread(img_loc + img_name)
  img = img[0:715, 0:1024]
  detect_and_draw_circles(blurred_img, dp, minDist, param1, param2, rmin, img, False)

  save_image(img)
    
  running = input("Run again with different parameters? (y/n): ")

Accuracy of the radii ± 17.5
Save the image? (y/n): n
Run again with different parameters? (y/n): y
Save the image? (y/n): n
Run again with different parameters? (y/n): y
Save the image? (y/n): n
Run again with different parameters? (y/n): y
Save the image? (y/n): n
Run again with different parameters? (y/n): y
Save the image? (y/n): n
Run again with different parameters? (y/n): n
